**Importar biliotecas y leer datos**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importamos los archivos y los almacenamos como DataFrames

df_steam_games = pd.read_csv('steam_games.csv')
df_reviews = pd.read_csv('reviews.csv')
df_items = pd.read_csv('items.csv')

**Extracción, transformación y carga de datos (ETL)**

**df_reviews**

In [3]:
# Visualizamos el dataframe
df_reviews.head()

,user_id,item_id,posted,recommend,review
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,js41637,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...


In [4]:
# Visualizamos las dimensiones del df
df_reviews.shape

(59305, 5)

In [5]:
# Visualizamos las columnas existentes
df_reviews.columns

Index(['user_id', 'item_id', 'posted', 'recommend', 'review'], dtype='object')

In [6]:
# Visualizamos el tipo de datos de las columnas
df_reviews.dtypes

user_id      object
item_id       int64
posted       object
recommend      bool
review       object
dtype: object

In [7]:
# Revisamos datos nulos por columna
datos_no_nulos = df_reviews.count()
datos_nulos = df_reviews.isnull().sum()

informacion_columnas = pd.DataFrame({'Datos no nulos': datos_no_nulos,'Datos nulos': datos_nulos})
informacion_columnas

,Datos no nulos,Datos nulos
user_id,59305,0
item_id,59305,0
posted,59305,0
recommend,59305,0
review,59275,30


In [8]:
# Realizamos el análisis de sentimiento para la columna 'review' de df_reviews_info

from textblob import TextBlob

# Función para analizar el sentimiento y asignar valores según la escala
def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    if analysis.sentiment.polarity < 0:
        return 0  # Sentimiento negativo
    elif analysis.sentiment.polarity == 0:
        return 1  # Sentimiento neutral
    else:
        return 2  # Sentimiento positivo

In [9]:
# Transformando tipo de dato float a str para la columna 'review' debido a que el análisis de sentimiento arrojaba un error por el tipo de dato
df_reviews['review'] = df_reviews['review'].astype(str)

In [10]:
# Aplicar la función de análisis de sentimiento a la columna 'texto' y crear 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analizar_sentimiento)

In [11]:
# Eliminamos la columna 'review'
df_reviews = df_reviews.drop('review', axis=1)

In [12]:
df_reviews.head()

,user_id,item_id,posted,recommend,sentiment_analysis
0,76561197970982479,1250,"Posted November 5, 2011.",True,2
1,76561197970982479,22200,"Posted July 15, 2011.",True,2
2,76561197970982479,43110,"Posted April 21, 2011.",True,2
3,js41637,251610,"Posted June 24, 2014.",True,2
4,js41637,227300,"Posted September 8, 2013.",True,0


In [13]:
# Transformamos el tipo de dato de posted para que sea en un formato de fecha

# Extraer la parte de la fecha de la columna "posted" usando una expresión regular
df_reviews['posted'] = df_reviews['posted'].str.extract(r'Posted (.+?)\.')

# Definir un formato que permita fechas con o sin año
date_formats = ['%B %d, %Y', '%B %d']

# Intentar convertir la columna "posted" al formato deseado
for date_format in date_formats:
    try:
        df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], format=date_format, errors='coerce')
        break  # Salir del bucle si la conversión es exitosa
    except ValueError:
        pass  # Continuar con el siguiente formato si hay un error

# Reemplazar el formato de fecha con el deseado
df_reviews['posted'] = df_reviews['posted'].dt.strftime('%Y-%m-%d')

**df_steam_games**

In [14]:
# Visualizamos el dataframe
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域
4,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",http://steamcommunity.com/app/772540/reviews/?...,"['Single-player', 'Steam Achievements']",3.99,False,772540.0,Trickjump Games Ltd


In [15]:
# Visualizamos las dimensiones del df
df_steam_games.shape

(28852, 13)

In [16]:
# Visualizamos las columnas existentes
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [17]:
# Visualizamos el tipo de datos de las columnas
df_steam_games.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price            object
early_access       bool
id              float64
developer        object
dtype: object

In [18]:
# Consideramos que columnas eliminar de df_steam_games

# app_name: Contiene los mismos datos que 'title'
# tags: contiene caracteristicas similares que 'genres'
# url: No contiene información que pueda aportar al modelo
# reviews_url: No posee información importante

In [19]:
# columnas_eliminar = ['app_name', 'tags', 'url', 'reviews_url']

# df_steam_games = df_steam_games.drop(columns=columnas_eliminar)

In [20]:
# Chequeamos las columnas que quedaron
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [21]:
# Revisamos datos nulos por columna

datos_no_nulos = df_steam_games.count()
datos_nulos = df_steam_games.isnull().sum()

informacion_columnas = pd.DataFrame({'Datos no nulos': datos_no_nulos,'Datos nulos': datos_nulos})
informacion_columnas

,Datos no nulos,Datos nulos
publisher,23936,4916
genres,28852,0
app_name,28851,1
title,28851,1
url,28852,0
release_date,28834,18
tags,28828,24
reviews_url,28851,1
specs,28566,286
price,27622,1230


In [22]:
# Elimino el unico registro con id nulo
# Eliminar registros con valores nulos en 'columna_especifica'
df_steam_games = df_steam_games.dropna(subset=['id'])

In [23]:
# Transformo 'id' en un tipo de dato int 
df_steam_games['id'] = df_steam_games['id'].astype(int)

In [24]:
# Visualizo los valores unicos en 'price' 
df_steam_games['price'].unique()

array(['4.99', 'Free To Play', 'Free to Play', '0.99', '3.99', '9.99',
       '18.99', '29.99', nan, '10.99', '2.99', '1.59', '14.99', '1.99',
       '59.99', '8.99', '6.99', '7.99', '39.99', 'Free', '19.99', '7.49',
       '12.99', '5.99', '2.49', '15.99', '1.25', '24.99', '17.99',
       '61.99', '3.49', '11.99', '13.99', 'Free Demo', 'Play for Free!',
       '34.99', '74.76', '1.49', '32.99', '99.99', '14.95', '69.99',
       '16.99', '79.99', '49.99', '5.0', '44.99', '13.98', '29.96',
       '109.99', '149.99', '771.71', 'Install Now', '21.99', '89.99',
       'Play WARMACHINE: Tactics Demo', '0.98', '139.92', '4.29', '64.99',
       'Free Mod', '54.99', '74.99', 'Install Theme', '0.89',
       'Third-party', '0.5', 'Play Now', '299.99', '1.29', '119.99',
       '3.0', '15.0', '5.49', '23.99', '49.0', '20.99', '10.93', '1.39',
       'Free HITMAN™ Holiday Pack', '36.99', '4.49', '2.0', '4.0', '1.95',
       '1.5', '199.0', '189.0', '6.66', '27.99', '129.99', '179.0',
       '26.99'

In [25]:
# Unifico todos los valores distintos hacia un unico valor 'Free To Play'
# Usa la función .replace() para realizar la transformación
df_steam_games['price'] = df_steam_games['price'].replace('Free to Play', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Free Demo', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Play for Free!', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Install Now', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Play WARMACHINE: Tactics Demo', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Free Mod', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Install Theme', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Third-party', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Play Now', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Free HITMAN™ Holiday Pack', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Play the Demo', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Free to Try', 'Free To Play')
df_steam_games['price'] = df_steam_games['price'].replace('Free to Use', 'Free To Play')

In [26]:
# Visualizo los valores unicos en 'price' 
df_steam_games['price'].unique()

array(['4.99', 'Free To Play', '0.99', '3.99', '9.99', '18.99', '29.99',
       nan, '10.99', '2.99', '1.59', '14.99', '1.99', '59.99', '8.99',
       '6.99', '7.99', '39.99', 'Free', '19.99', '7.49', '12.99', '5.99',
       '2.49', '15.99', '1.25', '24.99', '17.99', '61.99', '3.49',
       '11.99', '13.99', '34.99', '74.76', '1.49', '32.99', '99.99',
       '14.95', '69.99', '16.99', '79.99', '49.99', '5.0', '44.99',
       '13.98', '29.96', '109.99', '149.99', '771.71', '21.99', '89.99',
       '0.98', '139.92', '4.29', '64.99', '54.99', '74.99', '0.89', '0.5',
       '299.99', '1.29', '119.99', '3.0', '15.0', '5.49', '23.99', '49.0',
       '20.99', '10.93', '1.39', '36.99', '4.49', '2.0', '4.0', '1.95',
       '1.5', '199.0', '189.0', '6.66', '27.99', '129.99', '179.0',
       '26.99', '399.99', '31.99', '399.0', '20.0', '40.0', '3.33',
       '22.99', '320.0', '38.85', '71.7', '995.0', '27.49', '3.39', '6.0',
       '19.95', '499.99', '199.99', '16.06', '4.68', '131.4', '44.98',
 

In [28]:
# En la columna 'price' formalizo los valores a float
# Primero debo convertir los 'Free To Play' en 0
# Define una función para realizar la conversión
def convert_price(value):
    if value == 'Free To Play':
        return 0.0
    try:
        return float(value)
    except ValueError:
        return None  # Si no se puede convertir a float, devolvemos None

# Aplica la función a la columna 'price'
df_steam_games['price'] = df_steam_games['price'].apply(convert_price)

# Convierte la columna en tipo float
df_steam_games['price'] = df_steam_games['price'].astype(float)

In [29]:
# Transformo el nombre de la columna a 'item_id'
df_steam_games.rename(columns={'id': 'item_id'}, inplace=True)

In [30]:
# Visualizamos el tipo de datos de las columnas
df_steam_games.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price           float64
early_access       bool
item_id           int32
developer        object
dtype: object

In [31]:
# Revisamos datos nulos por columna

datos_no_nulos = df_steam_games.count()
datos_nulos = df_steam_games.isnull().sum()

informacion_columnas = pd.DataFrame({'Datos no nulos': datos_no_nulos,'Datos nulos': datos_nulos})
informacion_columnas

,Datos no nulos,Datos nulos
publisher,23935,4916
genres,28851,0
app_name,28850,1
title,28850,1
url,28851,0
release_date,28833,18
tags,28827,24
reviews_url,28851,0
specs,28565,286
price,26941,1910


**df_items**

In [32]:
# Visualizamos el dataframe
df_items.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


In [33]:
# Visualizamos las dimensiones del df
df_items.shape

(5153209, 5)

In [34]:
# Visualizamos las columnas existentes
df_items.columns

Index(['user_id', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')

In [35]:
# Visualizamos el tipo de datos de las columnas
df_items.dtypes

user_id             object
item_id              int64
item_name           object
playtime_forever     int64
playtime_2weeks      int64
dtype: object

In [37]:
# Convierto a'user_id' en tipo de dato string para falicitar luego su lectura
df_items['user_id'] = df_items['user_id'].apply(str)

In [38]:
df_items.dtypes

user_id             object
item_id              int64
item_name           object
playtime_forever     int64
playtime_2weeks      int64
dtype: object

In [39]:
# Revisamos datos nulos por columna

datos_no_nulos = df_items.count()
datos_nulos = df_items.isnull().sum()

informacion_columnas = pd.DataFrame({'Datos no nulos': datos_no_nulos,'Datos nulos': datos_nulos})
informacion_columnas

,Datos no nulos,Datos nulos
user_id,5153209,0
item_id,5153209,0
item_name,5153209,0
playtime_forever,5153209,0
playtime_2weeks,5153209,0


**Exportación**

In [40]:
# Con los dataframes transformados los exportamos en limpio para su posterior utilizacion

df_steam_games.to_csv('E:\\Henry Curso\\Curso\\LABS\\PI_MLOps\\FastAPI\\steam_games.csv', index=False)

In [41]:
df_reviews.to_csv('E:\\Henry Curso\\Curso\\LABS\\PI_MLOps\\FastAPI\\reviews.csv', index=False)

In [42]:
df_items.to_csv('E:\\Henry Curso\\Curso\\LABS\\PI_MLOps\\FastAPI\\items.csv', index=False)